# Notebook Examples for Chapter 6

In [1]:
import warnings
# these are innocuous but irritating
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
%matplotlib inline

## Training data separability

In [ ]:
import ee
ee.Initialize()

# first 4 principal components of ASTER image
image = ee.Image('users/mortcanty/supervisedclassification/AST_20070501_pca') \
            .select(0,1,2,3)

# training data
table = ee.FeatureCollection('users/mortcanty/supervisedclassification/train')
trainData = image.sampleRegions(table,['CLASS_ID'])
print trainData.size().getInfo()  

In [ ]:
def jmsep(class1,class2,image,table):
# Jeffries-Matusita separability    
    table1 = table.filter(
        ee.Filter.eq('CLASS_ID',str(class1-1)))
    m1 = image.reduceRegion(ee.Reducer.mean(),table1)\
              .toArray() 
    s1 = image.toArray() \
         .reduceRegion(ee.Reducer.covariance(),table1)\
         .toArray()
    table2 = table.filter(
        ee.Filter.eq('CLASS_ID',str(class2-1)))
    m2 = image.reduceRegion(ee.Reducer.mean(),table2)\
              .toArray()
    s2 = image.toArray() \
        .reduceRegion(ee.Reducer.covariance(),table2,15)\
              .toArray()
    m12 = m1.subtract(m2)  
    m12 = ee.Array([m12.toList()]) # makes 2D matrix  
    s12i = s1.add(s2).divide(2).matrixInverse()
#  first term in Bhattacharyya distance
    B1 = m12.matrixMultiply(
          s12i.matrixMultiply(m12.matrixTranspose())) \
            .divide(8)
    ds1 = s1.matrixDeterminant()
    ds2 = s2.matrixDeterminant() 
    ds12 = s1.add(s2).matrixDeterminant()
#  second term
    B2 = ds12.divide(2).divide(ds1.multiply(ds2).sqrt())\
             .log().divide(2)
    B = ee.Number(B1.add(B2).project([0]).toList().get(0))
#  J-M separability
    return ee.Number(1).subtract(ee.Number(1) \
             .divide(B.exp())) \
             .multiply(2)

In [ ]:
print jmsep(5,9,image,table).getInfo()

In [ ]:
def band_mean(current,prev):
    current = ee.String(current)
    prev = ee.Dictionary(prev)
    trainData = ee.FeatureCollection(prev.get('trainData'))
    class_id = prev.get('class_id')
    means = ee.List(prev.get('means'))
    mu = trainData.filter(ee.Filter.eq('CLASS_ID',class_id)).aggregate_mean(current)
    return ee.Dictionary({ 'trainData':trainData,'class_id':class_id,'means':means.add(mu) })

def class_mean(trainData,class_id,bandNames):
    first = ee.Dictionary({'trainData':trainData,'class_id':str(class_id),'means':ee.List([])})
    return ee.Dictionary(bandNames.iterate(band_mean,first)).get('means')

mu = ee.Array(class_mean(trainData,7,image.bandNames()))
print mu.getInfo()    

### Naive Bayes  on the GEE

In [ ]:
import IPython.display as disp
jet = 'black,blue,cyan,yellow,red,brown'

# rename the class ids from strings to integers
trainData = image.sampleRegions(table,['CLASS_ID'])\
    .remap(['0','1','2','3','4','5','6','7','8','9'],
           [0,1,2,3,4,5,6,7,8,9],'CLASS_ID')
    
# train a naive Bayes classifier    
classifier = ee.Classifier.continuousNaiveBayes()
trained = classifier\
    .train(trainData,'CLASS_ID',image.bandNames())

# classify the image and display    
classified = image.classify(trained)
url = classified.select('classification')\
    .getThumbURL({'min':0,'max':9,'palette':jet})
disp.Image(url=url)

### Bayes Maximum Likelihood

In [ ]:
run scripts/classify -p [1,2,3,4] -a 1 imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c \
-r  "['WATER', 'RAPESEED', 'SUGARBEET', 'SUBURBAN', 'INDUSTRIAL', 'CONIFEROUS', 'GRAIN', 'GRASSLAND', 'HERBIFEROUS', 'OPENCAST']" \
-s '/home/mort/LaTeX/new projects/CRC4/Chapter6/fig6_5.eps'

### Gaussian kernel

In [ ]:
run scripts/classify -p [1,2,3,4] -a 2 -P imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c \
-r "['WASSER [BL', 'RAPS [YELL', 'RUEBEN [CY', 'SIEDLUNG [', 'GEWERBE [M', 'NADELWALD', 'GETREIDE [', 'GRAS [RED2', 'LAUBWALD [', 'TAGEBAU [W']"

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_classprobs.tif -p [4,2,1] -e 1 \
#-s '/home/mort/LaTeX/new projects/CRC4/Chapter6/fig6_6.eps'

### Ffn with backpropagation

In [ ]:
run scripts/classify -p [1,2,3,4] -a 3 -e 10 -L [1000] imagery/AST_20070501_pca.tif imagery/train.shp

### Ffn with scaled conjugate gradiant

In [ ]:
run scripts/classify -p [1,2,3,4] -a 4 -e 1000 -L [10] imagery/AST_20070501_pca.tif imagery/train.shp

### Ffn with extended Kalman filter

In [ ]:
run scripts/classify -p [1,2,3,4] -a 5 -e 10 -L [10] imagery/AST_20070501_pca.tif imagery/train.shp

### Ffn in TensorFlow

#### Accessing the data

In [2]:
import auxil.readshp as rs
import gdal
import matplotlib.pyplot as plt
from osgeo.gdalconst import GA_ReadOnly
import numpy as np

# get the training data
infile='imagery/AST_20070501_pca.tif'
gdal.AllRegister()
inDataset = gdal.Open(infile,GA_ReadOnly)
pos=[1,2,3,4]
Gs,ls,K,_ = rs.readshp('imagery/train.shp',inDataset,pos)
m = ls.shape[0]
print '%i observations read'%m

# split into train/test
idx = np.random.permutation(m)
Gs = Gs[idx,:] 
ls = ls[idx,:]         
ls = np.argmax(ls,1)

Gstrn = Gs[:int(0.67*m),:]
lstrn = ls[:int(0.67*m)] 
Gstst = Gs[int(0.67*m):,:]  
lstst = ls[int(0.67*m):] 

# read entire image
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize  
Gs_all = np.zeros((cols*rows,4))
k= 0
for b in pos:
    band = inDataset.GetRasterBand(b)
    Gs_all[:,k] = band.ReadAsArray(0,0,cols,rows)\
                          .astype(float).ravel()
    k += 1      

reading training data...
7161 observations read


#### Network architecture

In [3]:
import tensorflow as tf
from datetime import datetime
# placeholders
Gs = tf.placeholder(tf.float32,shape=(None,4))
ls = tf.placeholder(tf.int64,shape=(None))
# hidden layer with rectified linear units (relu) 
hidden=tf.layers.dense(Gs,10,activation=tf.nn.relu)
# output layer
logits=tf.layers.dense(hidden,10)
# cross entropy cost function
xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits\
                            (labels=ls,logits=logits)
cost=tf.reduce_mean(xentropy)
# training algorithm with 0.01 learning rate
optimizer=tf.train.GradientDescentOptimizer(0.01)
training_op=optimizer.minimize(cost)
# variables initializer 
init=tf.global_variables_initializer()
# accuracy evaluation
correct=tf.nn.in_top_k(logits,ls,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
# saver
saver = tf.train.Saver()
# logger for tensorboard
cost_summary = tf.summary.scalar('COST',cost)

#### Training and testing

In [4]:
%%time
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = 'tf_logs/run-'+str(now)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
with tf.Session() as sess:
    init.run()
    for epoch in range(5000):
        if epoch % 200 ==0:
            summary_str =cost_summary.eval(feed_dict={Gs:Gstrn,ls:lstrn})
            file_writer.add_summary(summary_str,epoch)
        sess.run(training_op,feed_dict={Gs:Gstrn,ls:lstrn})
    acc = accuracy.eval(feed_dict={Gs:Gstst,ls:lstst})
    save_path = saver.save(sess,'imagery/dnn.ckpt')
file_writer.close()
print 'Test accuracy: %f'%acc

Test accuracy: 0.909898
CPU times: user 21.8 s, sys: 852 ms, total: 22.7 s
Wall time: 14.7 s


In [5]:
!tensorboard --logdir tf_logs/

TensorBoard 1.12.0 at http://vivo:6006 (Press CTRL+C to quit)
^C


#### Prediction

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'imagery/dnn.ckpt')
    Z = logits.eval(feed_dict={Gs:Gs_all})
    cls = np.argmax(Z,1)  

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.reshape(cls/10.0,(rows,cols)),cmap='jet')

### Deep learning network

In [2]:
run scripts/classify -p [1,2,3,4] -a 6 -e 1000 -L [10,10,10] imagery/AST_20070501_pca.tif imagery/train.shp

Training with Dnn(tensorflow)
reading training data...
training on 4797 pixel vectors...
classes: ['WASSER [BL', 'RAPS [YELL', 'RUEBEN [CY', 'SIEDLUNG [', 'GEWERBE [M', 'NADELWALD', 'GETREIDE [', 'GRAS [RED2', 'LAUBWALD [', 'TAGEBAU [W']
elapsed time 98.6261298656
classifying...
row: 0
row: 100
row: 200
row: 300
row: 400
row: 500
row: 600
row: 700
row: 800
row: 900
elapsed time 6.4888958931
thematic map written to: imagery/AST_20070501_pca_class.tif
test results written to: imagery/AST_20070501_pca_Dnn(tensorflow).tst
done


In [3]:
run scripts/ct imagery/AST_20070501_pca_Dnn(tensorflow).tst

classification statistics
Dnn(tensorflow)test results for imagery/AST_20070501_pca.tif
Sun Dec 30 12:40:49 2018
Classification image: imagery/AST_20070501_pca_class.tif
Class probabilities image: None

Misclassification rate: 0.033841
Standard deviation: 0.003719
Conf. interval (95 percent): [0.027275 , 0.041920]
Kappa coefficient: 0.961801
Standard deviation: 0.004199
Contingency Table
[[ 191.       0.       0.       0.       0.       0.       0.       0.       0.       0.     191.       1.   ]
 [   0.     156.       0.       0.       0.       0.       0.       0.       0.       0.     156.       1.   ]
 [   0.       0.     262.       0.       0.       0.       0.       0.       0.       1.     263.       0.996]
 [   0.       0.       0.     285.      12.       0.       0.       4.       0.       0.     301.       0.947]
 [   0.       0.       0.       9.     225.       0.       0.       0.       0.       0.     234.       0.962]
 [   0.       0.       0.       0.       0.     122.   

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c

In [ ]:
run scripts/ct imagery/AST_20070501_pca_Dnn(tensorflow).tst

### Support Vector Machine

In [ ]:
run scripts/classify -p [1,2,3,4] -a 7 imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
run scripts/ct imagery/AST_20070501_pca_SVM.tst

In [ ]:
# train a SVM  
classifier = ee.Classifier.svm(kernelType='RBF',gamma=0.01,cost=100)
trained = classifier.\
    train(trainData,'CLASS_ID',image.bandNames())

# classify the image and display    
classified = image.classify(trained)
url = classified.select('classification').\
    getThumbURL({'min':0,'max':9,'palette':jet})
disp.Image(url=url)